<a href="https://colab.research.google.com/github/Fausta-Tata/healthcare-ai-classification-summarization/blob/main/Capstone_Project_Hacktiv_Healthcare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain_community
!pip install replicate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 3.4 MB/s eta 0:00:00


In [2]:
# Setup Library
import pandas as pd
import json
import os
from langchain_community.llms import Replicate
from google.colab import userdata
import textwrap

In [3]:
# Set the API token
api_token = userdata.get('api_token')
os.environ["REPLICATE_API_TOKEN"] = api_token

In [4]:
# URL Github Healthcare Dataset

url = 'https://github.com/Fausta-Tata/healthcare-ai-classification-summarization/raw/refs/heads/main/data/HealthcareDocumentationDatabase.csv'


In [5]:
# Set model and model parameters
model = "ibm-granite/granite-3.3-8b-instruct"

parameters = {
  "temperature": 0.1,
  "top_p": 0.75,
  "max_tokens": 70,
  "repetition_penalty": 1.5,

}

In [6]:
llm = Replicate(
    model = model,
    replicate_api_token=api_token,
    model_kwargs = parameters
)

In [7]:
def analyze_transcription(transcription: str, specialty_list: str):
    prompt = f"""
    You are an expert medical assistant. Analyze the following medical transcription.

    Transcription:
    ---
    {transcription}
    ---

    Tasks:
    1. Generate a very concise "description" (maximum 20 word in clinical note style) summarizing the patient's condition.
    2. Identify the most relevant "medical_specialty". You MUST select exactly ONE specialty from this list: {specialty_list}

    Output:
    Return the result strictly in valid JSON format, like:

    {{
      "description": "...",
      "medical_specialty": "..."
    }}
    """

    try:
        # Invoke the model
        response = llm.invoke(prompt, parameters=parameters)

        # Cleaner and more robust JSON parsing
        start_index = response.find('{')
        end_index = response.rfind('}') + 1
        if start_index == -1 or end_index == 0:
            raise json.JSONDecodeError("No JSON object found in the response.", response, 0)

        json_str = response[start_index:end_index]
        json_response = json.loads(json_str)

        return {
            "description": json_response.get("description", "N/A"),
            "medical_specialty": json_response.get("medical_specialty", "N/A")
        }
    except json.JSONDecodeError as e:
        print(f"\nFailed to parse JSON. Error: {e}")
        return None
    except Exception as e:
        print(f"\nError: {e}")
        return None

In [8]:
def main():
    try:
        # Import Healthcare Dataset
        df = pd.read_csv(url)
        # Drop Null Values
        df.dropna(subset=['transcription', 'medical_specialty', 'description'], inplace=True)
    except Exception as e:
        print(f"Error: {e}")
        return

    # Create a Unique List of Medical Specialties for The Models to Choose From.
    unique_specialties = df['medical_specialty'].unique().tolist()
    specialty_list_str = ", ".join(f"'{s.strip()}'" for s in unique_specialties)

    # Taking Random Data Samples
    sample_size = 5
    sample_df = df.sample(n=sample_size, random_state=27)


    # Create Variable for Accuration Process
    total_sampel = 0
    prediksi_benar = 0

    # Analysis Process
    for index, row in sample_df.iterrows():
        print(f"\n---------------- Analisis Data #{index + 1} ----------------")
        original_transcription = row['transcription']
        original_specialty = row['medical_specialty'].strip()
        original_description = row['description'].strip()

        # Call Function AI
        ai_result = analyze_transcription(original_transcription, specialty_list_str)

        if ai_result:
            total_sampel += 1
            ai_specialty = ai_result['medical_specialty'].strip()
            ai_description = ai_result['description'].strip()

            print("\n--- Perbandingan Deskripsi ---")
            print(f"Deskripsi Asli: {textwrap.fill(original_description, width=100)}")
            print(f"\nDeskripsi AI  : {textwrap.fill(ai_description, width=100)}")

            print("\n\n--- Perbandingan Spesialisasi Medis ---")
            print(f"Spesialisasi Asli: {original_specialty}")
            print(f"Spesialisasi AI  : {ai_specialty}")

            # Logic of comparison of AI Results and Original Data
            if original_specialty.lower() in ai_specialty.lower():
                prediksi_benar += 1
                print("Status           : Akurat")
            else:
                print("Status           : Tidak Akurat")


    # Final Accuracy Result
    if total_sampel > 0:
        akurasi = (prediksi_benar / total_sampel) * 100
        print(f"\n\n------ HASIL ANALISIS KESELURUHAN ------")
        print(f"Total Sampel Dianalisis: {total_sampel}")
        print(f"Jumlah Prediksi Benar  : {prediksi_benar}")
        print(f"Tingkat Akurasi Model  : {akurasi:.2f}%")
    else:
        print("\nTidak ada sampel yang berhasil dianalisis.")



In [9]:
# Run The Program
if __name__ == "__main__":
    main()


---------------- Analisis Data #1824 ----------------

--- Perbandingan Deskripsi ---
Deskripsi Asli: A 19-year-old right-handed male injured in a motor vehicle accident.

Deskripsi AI  : 19-year-old with post-traumatic cervical & lumbar radiculopathy, disc herniations, myofascitis,
scheduled for EMG study.


--- Perbandingan Spesialisasi Medis ---
Spesialisasi Asli: Orthopedic
Spesialisasi AI  : Neurosurgery
Status           : Tidak Akurat

---------------- Analisis Data #501 ----------------

--- Perbandingan Deskripsi ---
Deskripsi Asli: Right middle ear exploration with a Goldenberg TORP reconstruction.

Deskripsi AI  : Profound mixed sensorineural conductive hearing loss, otosclerosis, stapes fixation, no round window
niche, revision surgery with Goldenberg torp reconstruction.


--- Perbandingan Spesialisasi Medis ---
Spesialisasi Asli: Surgery
Spesialisasi AI  : Otolaryngology
Status           : Tidak Akurat

---------------- Analisis Data #177 ----------------

--- Perbandinga